# 1. Setup

In [3]:
import os
import pyodbc
import pandas as pd


# 2. Connect

In [4]:
# Kết nối SQL Server
def create_sqlserver_connection(server='HTP-178\\HTP178', db_name='CarSales2021'):
    try:
        conn = pyodbc.connect(
            driver='{ODBC Driver 17 for SQL Server}',
            server=server,
            database=db_name,
            trusted_connection='yes'
        )
        print(f"Kết nối đến SQL Server ({db_name}) thành công!")
        return conn
    except Exception as e:
        print(f"Không thể kết nối đến SQL Server: {e}")
        return None

conn = create_sqlserver_connection()
cursor = conn.cursor()


Kết nối đến SQL Server (CarSales2021) thành công!


# 3. Create DB

# 4. Create Table

In [5]:
sql_file_path = r"D:\HTP\Project-personal\PriceCar\database\db.sql"

try:
    cursor.execute("SELECT COUNT(*) FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_NAME = 'fact_car_sale'")
    table_exists = cursor.fetchone()[0]

    if table_exists == 0:  # Nếu bảng chưa tồn tại, chạy file SQL
        with open(sql_file_path, "r", encoding="utf-8") as f:
            sql_script = f.read()

        for statement in sql_script.split(";"):
            if statement.strip():
                cursor.execute(statement)
        conn.commit()
        print("Tạo bảng thành công!")
    else:
        print("Bảng fact_car_sale đã tồn tại, sẽ xóa và tạo lại bảng")

        # Xóa bảng fact_car_sale trước
        cursor.execute("DROP TABLE IF EXISTS fact_car_sale")
        conn.commit()
        print("Đã xóa bảng fact_car_sale.")

        # Xóa các bảng còn lại sau
        cursor.execute("DROP TABLE IF EXISTS dim_car_model")
        cursor.execute("DROP TABLE IF EXISTS dim_car_type")
        cursor.execute("DROP TABLE IF EXISTS dim_color")
        cursor.execute("DROP TABLE IF EXISTS dim_poster")
        cursor.execute("DROP TABLE IF EXISTS dim_car_condition")
        conn.commit()
        print("Đã xóa các bảng còn lại.")

        # Tạo lại các bảng từ file SQL
        with open(sql_file_path, "r", encoding="utf-8") as f:
            sql_script = f.read()

        for statement in sql_script.split(";"):
            if statement.strip():
                cursor.execute(statement)
        conn.commit()
        print("Tạo bảng mới thành công!")

except Exception as e:
    print(f"Lỗi khi tạo bảng: {e}")


Bảng fact_car_sale đã tồn tại, sẽ xóa và tạo lại bảng
Đã xóa bảng fact_car_sale.
Đã xóa các bảng còn lại.
Tạo bảng mới thành công!


# 5. Insert data

In [6]:
# Danh sách file CSV chứa dữ liệu
CSV_FILES = {
    "dim_car_model": r"D:\HTP\Project-personal\PriceCar\dataset\processed\Dim_Car_Model.csv",
    "dim_car_type": r"D:\HTP\Project-personal\PriceCar\dataset\processed\Dim_Car_Type.csv",
    "dim_color": r"D:\HTP\Project-personal\PriceCar\dataset\processed\Dim_Color.csv",
    "dim_poster": r"D:\HTP\Project-personal\PriceCar\dataset\processed\Dim_Poster.csv",
    "dim_car_condition": r"D:\HTP\Project-personal\PriceCar\dataset\processed\Dim_Car_Condition.csv",
    "fact_car_sale": r"D:\HTP\Project-personal\PriceCar\dataset\processed\Fact_Car_Sale.csv"
}

# Hàm insert data từ CSV vào SQL Server
def insert_data_from_csv(conn, table_name, csv_path):
    try:
        df = pd.read_csv(csv_path)
        cursor = conn.cursor()

        # Tạo câu lệnh INSERT
        columns = ', '.join(df.columns)
        placeholders = ', '.join(['?'] * len(df.columns))
        insert_query = f"INSERT INTO {table_name} ({columns}) VALUES ({placeholders})"

        # Chèn từng dòng dữ liệu
        for row in df.itertuples(index=False, name=None):
            cursor.execute(insert_query, row)

        conn.commit()
        print(f"Đã insert dữ liệu vào {table_name} từ {csv_path}")
    except Exception as e:
        print(f"Lỗi insert vào {table_name}: {e}")

# Insert data vào các bảng
for table, csv_file in CSV_FILES.items():
    if os.path.exists(csv_file):
        insert_data_from_csv(conn, table, csv_file)
    else:
        print(f"File CSV {csv_file} không tồn tại!")

# Đóng kết nối SQL Server
cursor.close()
conn.close()

Đã insert dữ liệu vào dim_car_model từ D:\HTP\Project-personal\PriceCar\dataset\processed\Dim_Car_Model.csv
Đã insert dữ liệu vào dim_car_type từ D:\HTP\Project-personal\PriceCar\dataset\processed\Dim_Car_Type.csv
Đã insert dữ liệu vào dim_color từ D:\HTP\Project-personal\PriceCar\dataset\processed\Dim_Color.csv
Đã insert dữ liệu vào dim_poster từ D:\HTP\Project-personal\PriceCar\dataset\processed\Dim_Poster.csv
Đã insert dữ liệu vào dim_car_condition từ D:\HTP\Project-personal\PriceCar\dataset\processed\Dim_Car_Condition.csv
Đã insert dữ liệu vào fact_car_sale từ D:\HTP\Project-personal\PriceCar\dataset\processed\Fact_Car_Sale.csv
